In [22]:
import pandas as pd
import numpy as np
from collections import Counter


In [23]:
data = pd.read_csv('small_train.tsv', sep='\t')
data.head()

,chest_pain,thalassemia,heart_disease
0,0,1,1
1,1,0,0
2,0,0,1
3,0,0,1
4,1,0,0


In [24]:
labels = data['heart_disease']
values, counts = np.unique(labels, return_counts=True)
values, counts

(array([0, 1]), array([14, 14]))

In [ ]:
# get H(y)


In [ ]:
# attribute 1

In [37]:
# get probability of each value in attribute 1
values, indices, counts = np.unique(data.iloc[:, 0], return_inverse = True, return_counts = True)
prob_neg = counts[0] / len(indices)
prob_pos = 1 - prob_neg
prob_neg, prob_pos, indices

(0.5714285714285714,
 0.4285714285714286,
 array([0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 1, 0, 1, 0]))

In [30]:
# get H(y|x)

# when x1 = 0
y_neg_x0 = []
y_pos_x0 = []
y_neg_x1 = []
y_pos_x1 = []

for idx in range(len(indices)):
    if indices[idx] == 0:
        y_neg_x0.append(labels[idx])
    else:
        y_pos_x0.append(labels[idx])

count_x0 = 0
for i in y_neg_x0:
    if i == 0:
        count_x0 += 1
    
prob_x0_y0 = count / len(y_neg_x0)
prob_x0_y1 = 1 - prob_x0_y0
entropy_y_x0 = -prob_x0_y0 * np.log2(prob_x0_y0) - prob_x0_y1 * np.log2(prob_x0_y1)

# when x1 = 1
for idx in range(len(indices)):
    if indices[idx] == 1:
        y_neg_x1.append(labels[idx])
    else:
        y_pos_x1.append(labels[idx])

count_x1 = 0
for i in y_neg_x1:
    if i == 0:
        count_x1 += 1
    
prob_x1_y0 = count_x1 / len(y_neg_x1)
prob_x1_y1 = 1 - prob_x1_y0
entropy_y_x1 = -prob_x1_y0 * np.log2(prob_x1_y0) - prob_x1_y1 * np.log2(prob_x1_y1)




In [31]:
cond_entropy = prob_neg * entropy_y_x0 + prob_pos * entropy_y_x1
cond_entropy

0.7421685375402277

In [30]:
from decision_tree import Prep
from inspection import get_entropy

attr_num = data.shape[1] - 1
mi_ls = []
for x in range(attr_num):
    prob_neg, prob_pos, indices = Prep.calc_prob(data, x)
    cond_entropy = Prep.calc_conditional_entropy(indices, prob_neg, prob_pos, labels)
    entropy = get_entropy(labels)
    mi = Prep.calc_mi(entropy, cond_entropy)
    if mi > 0:
        mi_ls.append(mi)
    
max_mi = max(mi_ls)
# It is possible for different columns to have equal values for mutual information. 
# In this case, you should split on the first column to break ties
max_mi_attr_idx = mi_ls.index(max_mi)
max_mi_attr_idx
            


0

In [31]:
from decision_tree import best_split

ImportError: cannot import name 'best_split' from 'decision_tree' (/Users/anyxling/Downloads/cmu_10601_hws/hw2/handout/decision_tree.py)

In [26]:
best_attr_idx = best_split(data, labels)
best_attr_idx

NameError: name 'best_split' is not defined